## 常用计量经济学方法 —— Python实现

## 线性回归模型的OLS估计

### statsmodels文档中的示例程序

我们可以先来看[statsmodels文档中的示例程序](http://www.statsmodels.org/stable/gettingstarted.html)。首先导入相关的模块与函数。

>**主要相关模块说明**

> **[Statsmodels](http://www.statsmodels.org/)** 是一个Python的第三方模块，他封装了许多计量模型，方便学者直接调用。我们主要应用这个模块进行计量分析。

> **[pasty](https://patsy.readthedocs.io/en/latest/)** is a Python library for describing statistical models (especially linear models, or models that have a linear component) and building design matrices. Patsy brings the convenience of [R](http://www.r-project.org/) "formulas" to Python.

In [1]:
import os
import patsy
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

然后载入和清洗数据。

In [2]:
df = pd.read_csv("https://p193.p3.n0.cdn.getcloudapp.com/items/qGuoNwBg/Guerry.csv")    # 载入csv格式的数据文件
vars = ['Department', 'Lottery', 'Literacy', 'Wealth', 'Region']    # 选择感兴趣的变量
df = df[vars]    # 选择相关数据列
df = df.dropna()    # 剔除缺失值
df[-5:]

,Department,Lottery,Literacy,Wealth,Region
80,Vendee,68,28,56,W
81,Vienne,40,25,68,W
82,Haute-Vienne,55,13,67,C
83,Vosges,14,62,82,E
84,Yonne,51,47,30,C


这时候**df**变量实际上就是我们回归所用的数据集。我们知道由于OLS估计量的公式是

$$\hat{\beta}=\left(X^{\prime} X\right)^{-1} X^{\prime} y$$

所以，可以用**patsy**模块中的**dmatrices**函数或者其中的$X$和$y$。

> 函数dmatrices的作用
>
> - 它会把类别变量分解成一系列的虚拟变量
> - 它会在$X$矩阵中添加常数列
> - 它会返回pandas的DataFrames对象

In [3]:
y, X = patsy.dmatrices('Lottery ~ Literacy + Wealth + Region', data=df, return_type='dataframe')
print(y[:3])
print(X[:3])

   Lottery
0     41.0
1     38.0
2     66.0
   Intercept  Region[T.E]  Region[T.N]  Region[T.S]  Region[T.W]  Literacy  \
0        1.0          1.0          0.0          0.0          0.0      37.0   
1        1.0          0.0          1.0          0.0          0.0      51.0   
2        1.0          0.0          0.0          0.0          0.0      13.0   

   Wealth  
0    73.0  
1    22.0  
2    61.0  


In [4]:
mod = sm.OLS(y, X)    # Describe model
res = mod.fit()       # Fit model
print(res.summary()) 

                            OLS Regression Results                            
Dep. Variable:                Lottery   R-squared:                       0.338
Model:                            OLS   Adj. R-squared:                  0.287
Method:                 Least Squares   F-statistic:                     6.636
Date:                Wed, 11 Mar 2020   Prob (F-statistic):           1.07e-05
Time:                        14:54:59   Log-Likelihood:                -375.30
No. Observations:                  85   AIC:                             764.6
Df Residuals:                      78   BIC:                             781.7
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      38.6517      9.456      4.087      

### 伍德里奇书中的例子3.1

In [5]:
df = pd.read_stata("https://p193.p3.n0.cdn.getcloudapp.com/items/YEuAolLR/GPA1.DTA")    # 载入数据

In [6]:
results = smf.ols('colGPA ~ hsGPA + ACT', data=df).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                 colGPA   R-squared:                       0.176
Model:                            OLS   Adj. R-squared:                  0.164
Method:                 Least Squares   F-statistic:                     14.78
Date:                Wed, 11 Mar 2020   Prob (F-statistic):           1.53e-06
Time:                        14:54:59   Log-Likelihood:                -46.573
No. Observations:                 141   AIC:                             99.15
Df Residuals:                     138   BIC:                             108.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.2863      0.341      3.774      0.0

### 计量经济学导论案例的Python实现

#### 案例2.3 首席执行官的薪水和股本回报率

In [7]:
df = pd.read_stata("https://p193.p3.n0.cdn.getcloudapp.com/items/nOuNkn77/CEOSAL1.DTA")    # 载入数据
results = smf.ols('salary ~ roe', data=df).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                 salary   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     2.767
Date:                Wed, 11 Mar 2020   Prob (F-statistic):             0.0978
Time:                        14:55:09   Log-Likelihood:                -1804.5
No. Observations:                 209   AIC:                             3613.
Df Residuals:                     207   BIC:                             3620.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    963.1913    213.240      4.517      0.0

In [8]:
# Salary for ROE = 0
print(results.params['Intercept'] + results.params['roe'] * 0)

# Salary for ROE = 30
print(results.params['Intercept'] + results.params['roe'] * 30)

963.191336472558
1518.226926829006


#### 案例2.6 CEO的薪水和股本回报率

In [2]:
df = pd.read_stata("https://p193.p3.n0.cdn.getcloudapp.com/items/nOuNkn77/CEOSAL1.DTA")    # 载入数据
results = smf.ols('salary ~ roe', data=df).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                 salary   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     2.767
Date:                Thu, 27 May 2021   Prob (F-statistic):             0.0978
Time:                        21:15:29   Log-Likelihood:                -1804.5
No. Observations:                 209   AIC:                             3613.
Df Residuals:                     207   BIC:                             3620.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    963.1913    213.240      4.517      0.0

In [10]:
df['salhat'] = results.predict()
df['uhat'] = df['salary'] - df['salhat']
print(df.loc[0:15,('roe', 'salary', 'salhat', 'uhat')])

          roe  salary       salhat        uhat
0   14.100000    1095  1224.058071 -129.058071
1   10.900000    1001  1164.854261 -163.854261
2   23.500000    1122  1397.969216 -275.969216
3    5.900000     578  1072.348338 -494.348338
4   13.800000    1368  1218.507712  149.492288
5   20.000000    1145  1333.215063 -188.215063
6   16.400000    1078  1266.610785 -188.610785
7   16.299999    1094  1264.760660 -170.760660
8   10.500000    1237  1157.453793   79.546207
9   26.299999     833  1449.772523 -616.772523
10  25.900000     567  1442.372056 -875.372056
11  26.799999     933  1459.023116 -526.023116
12  14.800000    1339  1237.008898  101.991102
13  22.299999     937  1375.767778 -438.767778
14  56.299999    2011  2004.808114    6.191886
15  12.600000    1585  1196.306291  388.693709


#### 案例4.4 校园犯罪与注册人数

In [11]:
df = pd.read_stata("https://p193.p3.n0.cdn.getcloudapp.com/items/RBudOLem/campus.dta")    # 载入数据
results = smf.ols('lcrime ~ lenroll', data=df).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                 lcrime   R-squared:                       0.585
Model:                            OLS   Adj. R-squared:                  0.580
Method:                 Least Squares   F-statistic:                     133.8
Date:                Wed, 11 Mar 2020   Prob (F-statistic):           7.83e-20
Time:                        14:55:10   Log-Likelihood:                -125.83
No. Observations:                  97   AIC:                             255.7
Df Residuals:                      95   BIC:                             260.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -6.6314      1.034     -6.416      0.0

使用[$t$检验](http://www.statsmodels.org/stable/generated/statsmodels.regression.linear_model.RegressionResults.t_test.html#statsmodels.regression.linear_model.RegressionResults.t_test)进行假设检验。

In [12]:
hypotheses = 'lenroll = 1'
t_test = results.t_test(hypotheses)
print(t_test)

                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0             1.2698      0.110      2.457      0.016       1.052       1.488


如果想要手动进行检验，可以利用[`scipy.stats`](https://docs.scipy.org/doc/scipy/reference/stats.html#)下的[t统计分布](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.t.html#scipy.stats.t)。

In [45]:
from scipy.stats import t

# std err for lenroll coef
se_lenroll = results.bse[1]
# calculate t value
t_lenroll = (results.params[1] - 1) / se_lenroll

print("T-statistics: {:.6}, P value: {:.6}".format(t_lenroll, 1 - t.cdf(t_lenroll, 120)))

T-statistics: 2.45737, P value: 0.00771259


#### 案例4.9 孩子出生体重方程中父母的受教育程度

In [47]:
df = pd.read_stata("http://fmwww.bc.edu/ec-p/data/wooldridge/bwght.dta")    # 载入数据
results = smf.ols('bwght ~ cigs + parity + faminc + motheduc + fatheduc', data=df).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  bwght   R-squared:                       0.039
Model:                            OLS   Adj. R-squared:                  0.035
Method:                 Least Squares   F-statistic:                     9.553
Date:                Wed, 11 Mar 2020   Prob (F-statistic):           5.99e-09
Time:                        16:46:30   Log-Likelihood:                -5242.2
No. Observations:                1191   AIC:                         1.050e+04
Df Residuals:                    1185   BIC:                         1.053e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    114.5243      3.728     30.716      0.0

F检验可以使用f_test函数。

In [55]:
hypotheses = '(motheduc = 0), (fatheduc = 0)'
f_test = results.f_test(hypotheses)

print("F-statistics: {:.4}, P value: {:.4}".format(f_test.fvalue[0][0], f_test.pvalue))

F-statistics: 1.437, P value: 0.238


#### 案例7.6 对数小时工资方程

In [57]:
df = pd.read_stata("http://fmwww.bc.edu/ec-p/data/wooldridge/wage1.dta")    # 载入数据
results = smf.ols('lwage ~ female * married + educ + exper + expersq + tenure + tenursq', data=df).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.461
Model:                            OLS   Adj. R-squared:                  0.453
Method:                 Least Squares   F-statistic:                     55.25
Date:                Wed, 11 Mar 2020   Prob (F-statistic):           1.28e-64
Time:                        17:12:05   Log-Likelihood:                -250.96
No. Observations:                 526   AIC:                             519.9
Df Residuals:                     517   BIC:                             558.3
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.3214      0.100      3.

In [60]:
df['male'] = 1 - df['female']
df['single'] = 1 - df['married']
results = smf.ols('lwage ~ married:male + female:married + single:female + educ + exper + expersq + tenure + tenursq', data=df).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.461
Model:                            OLS   Adj. R-squared:                  0.453
Method:                 Least Squares   F-statistic:                     55.25
Date:                Wed, 11 Mar 2020   Prob (F-statistic):           1.28e-64
Time:                        17:15:38   Log-Likelihood:                -250.96
No. Observations:                 526   AIC:                             519.9
Df Residuals:                     517   BIC:                             558.3
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.3214      0.100      3.